## Preparing Data

First, let's import the necessary Python modules.
In google colab you will need to install the some libraries.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets
!pip install transformers
!pip install seqeval
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 17.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 62.2 MB/s eta 0:00

In [3]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
import numpy as np
from datasets import load_metric,Dataset,DatasetDict,ClassLabel

In [5]:
model_checkpoint = 'xlm-roberta-base'
batch_size = 16

In [6]:
infile = open('/content/drive/MyDrive/FYP/Training_data.txt', 'r', encoding='utf-8-sig')
train = infile.readlines()
infile.close()

words = []
tags = []
for i in train:
    data = i.split("\n")[0]
    data = data.split("\t")
    words.append(data[0])
    tags.append(data[1])


In [7]:
vocabulary = []
for tag in tags:
  if tag not in vocabulary:
    vocabulary.append(tag)
vocab = {}
for i in range(len(vocabulary)):
  vocab[i] = vocabulary[i]
label_encoder = ClassLabel(names=vocabulary)
tags = label_encoder.str2int(tags)

In [8]:
sentences = []
pos_tags = []
index = 0
for i in range(len(words)):
  if words[i]=='۔':
    sentences.append(words[index:i+1])
    pos_tags.append(tags[index:i+1])
    index = i+1

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [10]:
init_token = tokenizer.cls_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, pad_token, unk_token)

<s> <pad> <unk>


In [11]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, pad_token_idx, unk_token_idx)

0 1 3


In [12]:
max_input_length = tokenizer.max_model_input_sizes[model_checkpoint]

print(max_input_length)

512


In [13]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(
    sentences, 
    pos_tags,
    test_size = 0.4
)

In [14]:
pos_train_dataset = Dataset.from_dict({'sentence':x_train,'tags':y_train})
pos_val_dataset = Dataset.from_dict({'sentence':x_val[:int(len(x_val)*0.5)],'tags':y_val[:int(len(y_val)*0.5)]})
pos_test_dataset = Dataset.from_dict({'sentence':x_val[int(len(x_val)*0.5):],'tags':y_val[int(len(y_val)*0.5):]})
dataset = DatasetDict({'train':pos_train_dataset,'validation':pos_val_dataset,'test':pos_test_dataset})


In [15]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [16]:
show_random_elements(dataset["train"])

,sentence,tags
0,"[میں, پیپلز, پارٹی, ،, بعض, مسلم, لیگ, , لڑائی, بیرونی, دہشت, گردوں, کے, خلاف, انکا, زیادہ, تر, کلام, صنف, غزل, میں, ،, ،, ،, ،, تاریخ, ،, سے, انکا, دیوان, خالی, نہیں, ۔]","[2, 0, 0, 4, 6, 1, 0, 4, 0, 1, 0, 0, 2, 1, 0, 6, 1, 0, 0, 0, 2, 4, 4, 4, 4, 0, 4, 2, 0, 0, 1, 16, 4]"
1,"[پہلے, پیسے, اور, حاضری, رجسٹر, لیکر, گئی, ۔]","[0, 0, 9, 1, 0, 0, 15, 4]"
2,"[ایک, اور, گیت, اُن, لوگوں, کے, نام, یاد, ۔]","[8, 9, 0, 18, 0, 2, 0, 0, 4]"
3,"[ریڈاروں, پر, سمندری, نقل, و, حرکت, اور, جہازوں, کی, آمد, جامد, دیکھی, جاسکتی, ہوگی, ۔]","[0, 2, 1, 0, 9, 0, 9, 0, 2, 0, 0, 5, 0, 0, 4]"
4,"[نے, ء, میں, نیوزی, لینڈ, کے, خلاف, میں, اپنے, کیریئر, کا, آغاز, سنچری, کے, ساتھ, اور, میدان, پر, اپنا, واں, آخری, ٹیسٹ, جنوبی, افریقہ, کے, خلاف, کھیلا, ۔]","[2, 31, 2, 1, 0, 2, 1, 2, 21, 0, 2, 0, 0, 2, 0, 9, 0, 2, 21, 0, 1, 0, 1, 10, 2, 1, 5, 4]"
5,"[جناب, ،, ابھی, صرف, اردو, اردو, ،, کہیں, کھو, ۔]","[0, 4, 0, 1, 10, 10, 4, 0, 5, 4]"
6,"[ہر, شے, کو, سمجھتا, :, میں, "", اسلامی, قوانین, یعنی, حدیث, پرستی, یا, "", کے, مطابق, قوانیں, بنانے, پر, پابندی, ۔]","[1, 0, 2, 5, 4, 2, 9, 1, 0, 9, 0, 5, 9, 9, 2, 0, 0, 11, 2, 0, 4]"
7,"[بعدازاں, بارک‌پور, ،, میرٹھ, ،, بریلی, اور, لکھنؤ, کے, سپاہی, ،, جنہوں, نے, فرنگی, تسلط, کے, خلاف, علم, جہاد, لہرایا, تھا, ،, آخر, انہی, اجداد, کے, فرزند, تھے, ۔]","[26, 10, 4, 10, 4, 10, 9, 10, 2, 0, 4, 3, 2, 1, 0, 2, 1, 0, 0, 5, 7, 4, 26, 14, 0, 2, 0, 5, 4]"
8,"[کھیل, میں, جیت, ھوتا, ھے, , مشاعرہ, ہم, خود, کروالیں, گے, ), ضرور, گا, کو, آپ, تایا, ۔]","[0, 2, 5, 5, 0, 4, 0, 18, 17, 0, 7, 4, 26, 7, 2, 18, 0, 4]"
9,"[وہ, اپنے, بادشاہ, سے, ٹوٹ, کر, محبت, کرتے, ہیں, ۔]","[18, 21, 0, 2, 5, 20, 0, 5, 7, 4]"


# Preprocessing Data

In [17]:
example = dataset["train"][2]
print(example["sentence"])

['خوشخبری', 'کہ', 'ابراہیم', '!', 'میں', 'تجھے', 'نسل', 'کی', 'امامت', 'عطا', 'کرتا', '۔']


In [18]:
tokenized_input = tokenizer(example["sentence"],is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['<s>', '▁خوش', 'خبر', 'ی', '▁کہ', '▁ابراہیم', '▁!', '▁میں', '▁تج', 'ھے', '▁نسل', '▁کی', '▁امام', 'ت', '▁عطا', '▁کرتا', '▁۔', '</s>']


In [19]:
len(example[f"tags"]), len(tokenized_input["input_ids"])

(12, 18)

In [20]:
print(tokenized_input.word_ids())

[None, 0, 0, 0, 1, 2, 3, 4, 5, 5, 6, 7, 8, 8, 9, 10, 11, None]


In [21]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

18 18


In [22]:
label_all_tokens = True

In [23]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["sentence"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [24]:
tokenize_and_align_labels(dataset['validation'][:5])

{'input_ids': [[0, 19486, 924, 17460, 1541, 8434, 499, 56809, 156815, 523, 38936, 212333, 39877, 160552, 6514, 124492, 3832, 105140, 27074, 504, 35493, 30981, 121027, 3067, 490, 8328, 146813, 523, 17248, 15035, 504, 15035, 34761, 66661, 490, 317, 175667, 3481, 2], [0, 64603, 4351, 317, 3481, 2], [0, 21142, 3481, 2], [0, 1541, 126189, 2567, 3067, 9446, 52, 10494, 39995, 33231, 2047, 907, 711, 52, 10469, 5042, 778, 4894, 490, 235318, 778, 216, 2103, 18296, 711, 216, 1533, 4647, 288, 504, 38734, 16634, 490, 4647, 216, 18900, 504, 31751, 90154, 92684, 490, 778, 33238, 1901, 3755, 554, 162889, 504, 120026, 118539, 1597, 490, 10469, 5042, 499, 20096, 43503, 7122, 76546, 368, 34895, 216, 2437, 5023, 2437, 3481, 2], [0, 36900, 8777, 4142, 907, 32711, 73077, 199989, 3481, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [25]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/25326 [00:00<?, ? examples/s]

Map:   0%|          | 0/8442 [00:00<?, ? examples/s]

Map:   0%|          | 0/8443 [00:00<?, ? examples/s]

In [26]:
tokenized_datasets["train"].features[f"tags"].feature = label_encoder
tokenized_datasets["validation"].features[f"tags"].feature = label_encoder
tokenized_datasets["test"].features[f"tags"].feature = label_encoder

In [27]:
label_list = tokenized_datasets["train"].features[f"tags"].feature.names
label_list

['NN',
 'JJ',
 'PSP',
 'PRR',
 'PU',
 'VBF',
 'Q',
 'AUXT',
 'CD',
 'CC',
 'NNP',
 'VBI',
 'AUXM',
 'VALA',
 'PDM',
 'AUXA',
 'NEG',
 'PRF',
 'PRP',
 'OD',
 'SCK',
 'APNA',
 'AUXP',
 'PRD',
 'SCP',
 'SC',
 'RB',
 'INJ',
 'PRE',
 'PRS',
 'FR',
 'SYM',
 'QM',
 'PRT',
 'FF',
 'pjak_uk',
 'index',
 'tests',
 'trackr',
 'AUXT ',
 'PSP ',
 'VBF ',
 '',
 'PU’']

In [28]:
metric = load_metric("seqeval")

<ipython-input-28-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [29]:
labels = [label_list[i] for i in example[f"tags"]]

metric.compute(predictions=[labels], references=[labels])

/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: SC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NNP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PU seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PSP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning:

{'BF': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'C': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'N': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 'NP': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'RP': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'SP': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'U': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [30]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [31]:
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {v: k for k, v in id2label.items()}

In [32]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint,id2label=id2label,
                                                        label2id=label2id, num_labels=len(label_list))

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-st

In [33]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-pos",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [34]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 277,486,892 trainable parameters


In [35]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [36]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/muhammadbilal/xlm-roberta-base-finetuned-pos into local empty directory.


Download file pytorch_model.bin:   0%|          | 8.00k/1.03G [00:00<?, ?B/s]

Download file runs/Mar18_12-35-51_353cc7c0397d/events.out.tfevents.1679143275.353cc7c0397d.208.2: 100%|#######…

Clean file runs/Mar18_12-35-51_353cc7c0397d/events.out.tfevents.1679143275.353cc7c0397d.208.2:  11%|#1        …

Download file runs/Mar16_04-25-48_d20e532ed874/events.out.tfevents.1678940755.d20e532ed874.214.5: 100%|#######…

...

Download file runs/Mar16_04-25-48_d20e532ed874/events.out.tfevents.1678946182.d20e532ed874.214.7: 100%|#######…

Clean file runs/Mar16_04-25-48_d20e532ed874/events.out.tfevents.1678946182.d20e532ed874.214.7: 100%|##########…

Clean file sentencepiece.bpe.model:   0%|          | 1.00k/4.83M [00:00<?, ?B/s]

Clean file tokenizer.json:   0%|          | 1.00k/16.3M [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/1.03G [00:00<?, ?B/s]

In [ ]:
!rm -rf /content/xlm-roberta-base-finetuned-pos

In [37]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.162700,0.128895,0.959926,0.963322,0.961621,0.965336
2,0.100900,0.093067,0.968024,0.971639,0.969828,0.972983
3,0.070500,0.076595,0.975848,0.977379,0.976613,0.978555
4,0.053600,0.069668,0.978695,0.979538,0.979117,0.981196
5,0.041900,0.070601,0.980046,0.980824,0.980435,0.982132


/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: SC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: JJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: P

TrainOutput(global_step=7915, training_loss=0.12150500858424965, metrics={'train_runtime': 3980.3552, 'train_samples_per_second': 31.814, 'train_steps_per_second': 1.989, 'total_flos': 4914800839937040.0, 'train_loss': 0.12150500858424965, 'epoch': 5.0})

In [38]:
trainer.evaluate()

{'eval_loss': 0.07060135900974274,
 'eval_precision': 0.9800463495146825,
 'eval_recall': 0.9808241257966878,
 'eval_f1': 0.9804350834037975,
 'eval_accuracy': 0.9821321165143382,
 'eval_runtime': 47.5657,
 'eval_samples_per_second': 177.481,
 'eval_steps_per_second': 11.1,
 'epoch': 5.0}

In [39]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'ALA': {'precision': 1.0,
  'recall': 0.9944289693593314,
  'f1': 0.9972067039106145,
  'number': 718},
 'B': {'precision': 0.9692685925015365,
  'recall': 0.9686732186732187,
  'f1': 0.9689708141321044,
  'number': 1628},
 'BF': {'precision': 0.976437239790947,
  'recall': 0.9746242263483643,
  'f1': 0.9755298907031285,
  'number': 11310},
 'BI': {'precision': 0.9866666666666667,
  'recall': 0.981500872600349,
  'f1': 0.984076990376203,
  'number': 2865},
 'C': {'precision': 0.9961857018308631,
  'recall': 0.9924003908370427,
  'f1': 0.9942894436286507,
  'number': 9211},
 'CK': {'precision': 0.96,
  'recall': 0.9795918367346939,
  'f1': 0.9696969696969697,
  'number': 196},
 'CP': {'precision': 0.8300653594771242,
  'recall': 0.9338235294117647,
  'f1': 0.878892733564014,
  'number': 136},
 'D': {'precision': 0.9720593308037254,
  'recall': 0.9794925269377824,
  'f1': 0.9757617728531857,
  'number': 2877},
 'DM': {'precision': 0.934640522875817,
  'recall': 0.9669421487603306,
  'f1

In [40]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/1.03G [00:00<?, ?B/s]

Upload file runs/Mar26_10-18-46_44d765b4665c/events.out.tfevents.1679830280.44d765b4665c.612.2:   0%|         …

Upload file runs/Mar26_10-18-46_44d765b4665c/events.out.tfevents.1679826252.44d765b4665c.612.0:   0%|         …

To https://huggingface.co/muhammadbilal/xlm-roberta-base-finetuned-pos
   1bad045..eda8b82  main -> main

   1bad045..eda8b82  main -> main

To https://huggingface.co/muhammadbilal/xlm-roberta-base-finetuned-pos
   eda8b82..a1b13ae  main -> main

   eda8b82..a1b13ae  main -> main



'https://huggingface.co/muhammadbilal/xlm-roberta-base-finetuned-pos/commit/eda8b82c1f11904c2669a9a6cbd8e2b2e680276c'